# OpenVINO Demo 3

## Let's detect objects *asynchronously* in real time using a cam feed!

Continuing on from the last demo, let's build upon what we learned and detect things in real time anhe d do things asynchronously! Spreading inference tasks upon multiple threads allows for us to use the full capability of our processor. While we wait on the inference task that we want to perhaps display, other inference tasks work in the background.

From the beginning, let's create functions that will help us out later. After we import all the libraries/packages we need, let's define our first function which will pre-process our image.

In [1]:
# Loading required packages
import cv2
import numpy as np
import matplotlib.pyplot as plt
from openvino.inference_engine import IENetwork
from openvino.inference_engine import IEPlugin
import time

%matplotlib inline

# CHANGE AS NEEDED
OS = 'linux'
dev = 'MYRIAD' # Change to MYRIAD if Intel NCS 2 plugged in

In [2]:
def pre_processing(obj_frame, input_shape):
    n, c, h, w = input_shape
    obj_in_frame = cv2.resize(obj_frame, (w, h))
    obj_in_frame = obj_in_frame.transpose((2, 0, 1))
    obj_in_frame = obj_in_frame.reshape((n, c, h, w))
    
    return {
        'blob' : obj_in_frame, 
        'frame': obj_frame, 
    }

Let's now define our second function which will return our net and useful information about it!

In [3]:
def construct_nn(path_to_xml, path_to_bin, dev, OS, reqs_num):
    obj_net = IENetwork(model=path_to_xml, weights=path_to_bin)
    input_layer = next(iter(obj_net.inputs))
    output_layer = next(iter(obj_net.outputs))
    net_shape = obj_net.inputs[input_layer].shape
    
    ext = None
    if OS.lower() == 'windows':
        ext = 'C:\\Users\\freyes\\Documents\\Intel\\OpenVINO\\inference_engine_samples_build\\intel64\\Release\\cpu_extension.dll'
    else:
        ext = '/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so'

    obj_plugin = IEPlugin(device=dev)
    if dev.lower() == 'cpu':
        obj_plugin.add_cpu_extension(ext)
        
    # ASYNC EDIT: Added request number instead of 1, like in sync demo
    obj_exec_net = obj_plugin.load(network=obj_net, num_requests=reqs_num)
    return {'net': obj_exec_net, 'input_layer': input_layer,
            'output_layer': output_layer, 'shape': net_shape}

Let's now define our third function which will process our data and draw the bounding box around the image we care about.

In [4]:
def draw_bb(obj_det, obj_frame):
    initial_w = obj_frame.shape[1]
    initial_h = obj_frame.shape[0]
    green = (0, 255, 0)

    for obj in obj_det[0][0]:
        # Draw only objects when probability more than specified threshold
        if obj[2] > 0.5:
            xmin = int(obj[3] * initial_w)
            ymin = int(obj[4] * initial_h)
            xmax = int(obj[5] * initial_w)
            ymax = int(obj[6] * initial_h)
            class_id = int(obj[1])

            # Draw box and label\class_id
            color = (min(class_id * 12.5, 255), min(class_id * 7, 255), min(class_id * 5, 255))
            cv2.rectangle(obj_frame, (xmin, ymin), (xmax, ymax), color, 2)
            cv2.putText(obj_frame, str(class_id), (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 0.8, green, 2, cv2.LINE_AA)
    

Finally, let's put the pieces together! In this example, more variables are added to keep track of async requests and their associated video frames. Over time, frames will fill in all requests.

In [5]:
# Object Detection Section but Async!
def main(OS, dev):
    # CHANGE THIS AS NEEDED
    reqs_num = 4 # request number that we pass in
    
    # ASYNC VARIABLES FOR MANAGING TASKS
    cur_request_id = reqs_num - 2
    next_request_id = reqs_num - 1
    requests_arr = [x for x in range(reqs_num)]
    frame_dicts = [None for x in range(reqs_num)]
    
    path_to_objxml = None
    path_to_objbin = None
    fp = 'fp32' if dev.lower() == 'cpu' else 'fp16'
    if OS.lower() == 'linux':
        path_to_objxml = 'gesture_optimized/' + fp + '/frozen_inference_graph.xml'
        path_to_objbin = 'gesture_optimized/' + fp + '/frozen_inference_graph.bin'
    elif os.lower() == 'windows':
        path_to_objxml = 'gesture_optimized\\' + fp + '\\frozen_inference_graph.xml'
        path_to_objbin = 'gesture_optimized\\' + fp + '\\frozen_inference_graph.bin'
    else:
        print("Need to have either linux or windows!")
        return
    
    # net, input_layer, output_layer, shape; added reqs_num
    net_dict = construct_nn(path_to_objxml, path_to_objbin, 'CPU', OS, reqs_num)
    obj_exec_net = net_dict['net']
    input_shape = net_dict['shape']
    input_layer = net_dict['input_layer']
    output_layer = net_dict['output_layer']
    
    vs = cv2.VideoCapture(0)
    
    # Adding a frame to the frame_dicts array
    # Corresponds to request id in requests_arr
    ret, pre_vframe = vs.read()
    frame_dicts[cur_request_id] = pre_processing(pre_vframe, input_shape)
    vframe = frame_dicts[cur_request_id]['frame']
    
    print("Click on the window and press q to exit the application.")
    while True:
        start = time.time()
        
        # Populating frame_dictss array with next video frame dict (next_image_dict)
        ret, next_vframe = vs.read()
        next_image_dict = pre_processing(next_vframe, input_shape)
        frame_dicts[next_request_id] = next_image_dict
        
        # Now will populate requests with next input blob, inference done in background
        next_vframe = next_image_dict['frame']
        next_inpBlob = next_image_dict['blob']
        obj_res = obj_exec_net.start_async(request_id=next_request_id, 
                                     inputs={input_layer: next_inpBlob})
        
        # Conditional will be used to process finished inference task
        if obj_exec_net.requests[cur_request_id].wait(-1) == 0:
            obj_res = obj_exec_net.requests[cur_request_id].outputs
            obj_detections = obj_res[output_layer]
            draw_bb(obj_detections, vframe)
        
        # Showing stats
        font = cv2.FONT_HERSHEY_SIMPLEX
        stamp = time.time() - start 
        cv2.putText(vframe, "Time: " + str(stamp), (30, 30), font, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(vframe, "FPS: " + str(1/stamp), (30, 60), font, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
        
        # Showing current request's associated frame that was post processed
        cv2.imshow("Frame (Async Mode)", vframe)

        # Switching requests and frames to focus on during next render
        cur_request_id, next_request_id = next_request_id, (next_request_id + 1) % reqs_num
        vframe = next_vframe
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
 
    # do a bit of cleanup
    cv2.destroyAllWindows()

main(OS, dev)

RuntimeError: Cannot load library '/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so': 126 from cwd: C:\Users\freyes\projects\wine_detector

# Congratulations!

We now know how to detect objects in real time!